In [1]:
import sys

sys.path.insert(0, "..")

import random
import numpy as np
from sklearn.preprocessing import LabelEncoder

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import torch
import pyro
import foraging_toolkit as ft
import torch.nn.functional as F
import pyro.distributions as dist
import pyro.optim as optim
from pyro.nn import PyroModule
from pyro.infer.autoguide import (
    AutoNormal,
    AutoDiagonalNormal,
    AutoMultivariateNormal,
    init_to_mean,
    init_to_value,
)
from pyro.contrib.autoguide import AutoLaplaceApproximation
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.infer import Predictive
from pyro.infer import MCMC, NUTS

import os
import logging

logging.basicConfig(format="%(message)s", level=logging.INFO)
smoke_test = "CI" in os.environ


# import foraging_toolkit as ft


[Günzel et al.](https://www.sciencedirect.com/science/article/pii/S2589004223004650?via%3Dihub#undtbl1) studied the balance between the use of social and personal information in foraging locust. Here we use one of the [associated datasets](https://zenodo.org/record/7541780), one with two equally high quality patches, Patch A located at (0.68074	-0.03068), and Patch B located at (-0.69292	-0.03068) - patch locations are in a separate .mat file, which can be accessed using Matlab or GNU Octave. The positions are normalized. 


In [2]:
with open("communicatorsDFs.pkl", "rb") as f:
    communicatorsDFs = pickle.load(f)
birdsDF = communicatorsDFs[0]
rewardsDF = communicatorsDFs[1]
birdsDF = birdsDF[birdsDF["bird"] >= 3]
indices_to_rename = birdsDF[birdsDF["bird"].isin([3, 4])].index
new_values = birdsDF.loc[indices_to_rename, "bird"].replace({3: 1, 4: 2})
birdsDF.loc[indices_to_rename, "bird"] = new_values
communicators = ft.object_from_data(birdsDF, rewardsDF)

rewardsDF.tail()


,x,y,time
1,0,25,42
0,0,24,43
1,0,25,43
0,0,25,44
0,0,25,45


In [3]:
locust = pd.read_csv("external_data/15EQ20191202_tracked.csv")

locust.drop("cnt", axis=1, inplace=True)
locust.rename(
    columns={"pos_x": "x", "pos_y": "y", "id": "bird", "frame": "time"}, inplace=True
)

locust = locust[["x", "y", "time", "bird"]]


encoder = LabelEncoder()

locust['bird'] = encoder.fit_transform(locust['bird'])

locust['bird'] = locust['bird'] + 1

locust.tail()

#print("min_time", locust['time'].min())

,x,y,time,bird
674995,-0.748473,-0.629429,45000,11
674996,0.661296,0.068313,45000,12
674997,-0.510189,0.663371,45000,13
674998,0.623918,0.022528,45000,14
674999,-0.891828,0.231515,45000,15


In [4]:
# we have positions at 25 frames per second, for 30 minutes
# we will thin this to 1 per second, obtaining 1800 frames
print("original_frames:", locust['time'].max())
print("original_shape:", locust.shape)


locust['time'] = np.round(locust['time']/ (45000/1800)).astype(int) + 1
locust = locust.drop_duplicates(subset=['time', 'bird'], keep='first')
locust = locust[locust['time'] <= 1800]
print("resulting_frames:", locust['time'].max())
print("resulting_shape:", locust.shape) #should be 8100, 4

print("min_time", locust['time'].min())
print("max_time", locust['time'].max())

original_frames: 45000
original_shape: (675000, 4)
resulting_frames: 1800
resulting_shape: (27000, 4)
min_time 1
max_time 1800


In [5]:
# coordinates are normalized wrt. radius,
# the diagonal of the arena is 90 cm
# body length of the locust is around 4 cm, so we will 
# satisfy ourselves with grid of half that size, 
# i.e. we want coordinates rounded and rescaled to 45x45 grid

def rescale_to_grid(column,size):
    mapped = (column + 1) / 2
    rescaled = mapped    
    rescaled = np.round(mapped * size - 1) + 1
    return rescaled

locust['x'] = rescale_to_grid(locust['x'], 45)
locust['y'] = rescale_to_grid(locust['y'], 45)
locust['type'] = "locust"

In [6]:
# we need to manually add rewards data frame
# Patch A located at (0.68074	-0.03068), and Patch B located at (-0.69292	-0.03068) 
# at all times

x = [0.68074, -0.69292]
y = [-0.03068, -0.03068]
time = list(range(1, locust['time'].max() + 1))

data = {'x': x * len(time), 'y': y * len(time), 'time': [t for t in time for _ in range(len(x))]}

rewardsDF = pd.DataFrame(data)
rewardsDF['x'] = rescale_to_grid(rewardsDF['x'], 45)
rewardsDF['y'] = rescale_to_grid(rewardsDF['y'], 45)

rewardsDF.head()

,x,y,time
0,38.0,22.0,1
1,7.0,22.0,1
2,38.0,22.0,2
3,7.0,22.0,2
4,38.0,22.0,3


In [7]:
# say we're now interested in the first three minutes
locust_initial = locust[locust['time'] <= 600]
rewards_initial = rewardsDF[rewardsDF['time'] <= 600]

In [8]:
# now massage this into our bird framework


def object_from_data(locustDF, rewardsDF, grid_size, frames):
   
    class EmptyObject:
        pass

    sim = EmptyObject()

    sim.grid_size = 45
    sim.num_frames = frames
    sim.birdsDF = locustDF
    sim.rewardsDF = rewardsDF
    sim.birds = [group for _, group in locustDF.groupby("bird")]
    sim.rewards = [group for _, group in rewardsDF.groupby("time")]
    sim.num_birds = len(sim.birds)

    step_maxes = []


    for b in range(len(sim.birds)):
        step_maxes.append(
            max(
                max(
                    [
                        abs(sim.birds[b]["x"].iloc[t + 1] - sim.birds[b]["x"].iloc[t])
                        for t in range(sim.num_frames - 1)
                    ]
                ),
                max(
                    [
                        abs(sim.birds[b]["y"].iloc[t + 1] - sim.birds[b]["y"].iloc[t])
                        for t in range(sim.num_frames - 1)
                    ]
                ),
            )
        )

    sim.step_size_max = max(step_maxes)

    return sim

loc_initial = object_from_data(locust_initial, rewards_initial, grid_size = 45, frames = 540)

loc = object_from_data(locust, rewardsDF, grid_size = 45, frames = 540)

In [9]:
loc.birdsDF.head()


,x,y,time,bird,type
0,31.0,41.0,1,1,locust
1,8.0,24.0,1,2,locust
2,40.0,27.0,1,3,locust
3,23.0,44.0,1,4,locust
4,18.0,39.0,1,5,locust


In [10]:
loc.birds[0]

,x,y,time,bird,type
0,31.0,41.0,1,1,locust
180,31.0,41.0,2,1,locust
555,31.0,41.0,3,1,locust
930,31.0,41.0,4,1,locust
1305,31.0,41.0,5,1,locust
...,...,...,...,...,...
672930,7.0,20.0,1796,1,locust
673305,7.0,20.0,1797,1,locust
673680,7.0,20.0,1798,1,locust
674055,7.0,20.0,1799,1,locust


In [11]:
#ft.animate_birds(loc_initial, plot_rewards=True, width=600, height=600, point_size=10)


In [39]:
rewards_decay=.2

#sim = loc_initial
sim = loc

tr = ft.rewards_to_trace(
        sim.rewards,
        sim.grid_size,
        sim.num_frames,
        rewards_decay,
    )

sim.tracesDF = tr["tracesDF"]

In [40]:
sim.visibility_range=10 
vis = ft.construct_visibility(
        sim.birds, sim.grid_size, visibility_range=sim.visibility_range
    )

sim.visibility = vis["visibility"]
sim.visibilityDF = vis["visibilityDF"]

In [41]:
getting_worse=1
optimal=2
proximity_decay=.5

prox = ft.generate_proximity_score(
        sim.birds,
        sim.visibility,
        visibility_range=sim.visibility_range,
        getting_worse=getting_worse,
        optimal=optimal,
        proximity_decay=proximity_decay,
    )

sim.proximityDF = prox["proximityDF"]

In [42]:
birds = sim.birds
step_size_max = sim.step_size_max
visibility_range = sim.visibility_range
how_far = sim.visibility.copy()

print(sim.num_frames)
print(sim.num_birds)

#t = 5

for b in range(sim.num_birds):
 #   print ( birds[b]["x"].iloc[t]) 
    for t in range(sim.num_frames - 1):
        x_new = int(birds[b]["x"].iloc[t + 1])
        y_new = int(birds[b]["y"].iloc[t + 1])

#     # print(how_far[b][t].head(n=1))
    _hf = how_far[b][t]
    _hf["how_far_squared"] = (_hf["x"] - x_new) ** 2 + (
        _hf["y"] - y_new
    ) ** 2
    _hf["how_far_squared_scaled"] = (
        -_hf["how_far_squared"]
        / (2 * (sim.step_size_max + sim.visibility_range) ** 2)
        + 1
    )
#     # print(how_far[b][t].head(n=1))

how_far[b][sim.num_frames - 1]["how_far_squared"] = np.nan
how_far[b][sim.num_frames - 1]["how_far_squared_scaled"] = np.nan

sim.how_far = how_far

birds_how_far = []
for bird in range(sim.num_birds):
    birds_how_far.append(pd.concat(how_far[bird]))

sim.how_farDF = pd.concat(birds_how_far)

540
15


In [43]:
sim.derivedDF = (
        sim.tracesDF.merge(sim.visibilityDF, how="inner")
        .merge(sim.proximityDF, how="inner")
        .merge(sim.how_farDF, how="inner")
    )

In [44]:
def prep_data_for_inference(sim_derived):

    df = sim_derived.derivedDF.dropna()

    data = torch.tensor(df[["proximity_standardized",
                         "trace_standardized", 
                         "visibility", "how_far_squared_scaled"]].values, dtype=torch.float32)

    proximity, trace, visibility, how_far = data[:, 0], data[:, 1], data[:, 2], data[:, 3]
    
    print(str(len(proximity)) + " data points prepared for inference.")
    return proximity, trace, visibility, how_far


proximity, trace, visibility, how_far_score = prep_data_for_inference(sim)

3911 data points prepared for inference.


In [45]:
proximity = (proximity - proximity.min()) / (proximity.max() - proximity.min())
trace = (trace - trace.min()) / (trace.max() - trace.min())
visibility = (visibility - visibility.min()) / (visibility.max() - visibility.min())

In [46]:

ft.visualise_bird_predictors(trace, proximity, how_far_score)


In [47]:
def summary(samples):
    site_stats = {}
    for site_name, values in samples.items():
        marginal_site = pd.DataFrame(values)
        describe = marginal_site.describe(percentiles=[.05, 0.25, 0.5, 0.75, 0.95]).transpose()
        site_stats[site_name] = describe[["mean", "std", "5%", "25%", "50%", "75%", "95%"]]
    return site_stats

In [48]:
# define the model
# p, t, v, b are the coefficients 
# for proximity, trace, visibility, and the intercept

# ps, ts, vs, bs are analogous coefficients,
# but they contribute to the variance,
# which is not assumed to remain fixed


def model_sigmavar(proximity, trace, visibility, how_far_score):
    p = pyro.sample("p", dist.Normal(0, .3))
    t = pyro.sample("t", dist.Normal(0, .3))
    v = pyro.sample("v", dist.Normal(0, .3))
    b = pyro.sample("b", dist.Normal(.5, .3))

    ps = pyro.sample("ps", dist.Normal(0, .3))
    ts = pyro.sample("ts", dist.Normal(0, .3))
    vs = pyro.sample("vs", dist.Normal(0, .3))
    bs = pyro.sample("bs", dist.Normal(.2, .3))


    sigmaRaw = bs + ps * proximity + ts * trace + vs * visibility
    sigma = pyro.deterministic("sigma", F.softplus(sigmaRaw))
    mean = b + p * proximity + t * trace + v * visibility

    with pyro.plate("data", len(how_far_score)):
        pyro.sample("obs", dist.Normal(mean, sigma), obs=how_far_score)

#pyro.render_model(model_sigmavar, model_args=(proximity, trace, visibility, how_far_score),
#                        render_distributions=True)

In [49]:
# Inference with SVI
# note how long this takes
# and compare time with MCM 

guide = AutoMultivariateNormal(model_sigmavar, init_loc_fn=init_to_mean)
svi = SVI(model_sigmavar,
        guide,
        optim.Adam({"lr": .01}),
        loss=Trace_ELBO())

iterations = []
losses = []

pyro.clear_param_store()
num_iters = 3000
for i in range(num_iters):
    elbo = svi.step(proximity, trace, visibility, how_far_score)
    iterations.append(i)
    losses.append(elbo)
    if i % 200 == 0:
        logging.info("Elbo loss: {}".format(elbo))


2023-08-24 16:35:11,906:  Elbo loss: 2855.6673089265823
2023-08-24 16:35:13,256:  Elbo loss: -5058.006201267242
2023-08-24 16:35:14,531:  Elbo loss: -8554.204847872257
2023-08-24 16:35:15,770:  Elbo loss: -8381.572995483875
2023-08-24 16:35:16,942:  Elbo loss: -10316.128569960594
2023-08-24 16:35:18,171:  Elbo loss: -10604.729516029358
2023-08-24 16:35:19,438:  Elbo loss: -10777.261811256409
2023-08-24 16:35:20,674:  Elbo loss: -10459.662950575352
2023-08-24 16:35:21,967:  Elbo loss: -10703.925302267075
2023-08-24 16:35:23,210:  Elbo loss: -9734.483172893524
2023-08-24 16:35:24,381:  Elbo loss: -10988.81010210514
2023-08-24 16:35:25,519:  Elbo loss: -11021.183928191662
2023-08-24 16:35:26,789:  Elbo loss: -10970.375266551971
2023-08-24 16:35:27,927:  Elbo loss: -10947.276655733585
2023-08-24 16:35:29,142:  Elbo loss: -11016.383594036102


In [51]:
# inspect the summary of the SVI posterior
# of key interest: t and p

num_samples = 1000
predictive = Predictive(model_sigmavar, guide=guide, 
                        num_samples=num_samples,
                        return_sites = ["t", "p"])
locust_sigmavar = {k: v.flatten().reshape(num_samples, -1).detach().cpu().numpy()
            for k, v in predictive(proximity, trace, visibility, how_far_score).items()
            if k != "obs"}

for site, values in summary(locust_sigmavar).items():
            print("Site: {}".format(site))
            print(values, "\n")

Site: p
       mean       std        5%       25%       50%       75%      95%
0  0.013883  0.007591  0.001506  0.008603  0.013856  0.018967  0.02603 

Site: t
       mean       std        5%      25%       50%       75%       95%
0  0.002383  0.003347 -0.003234  0.00004  0.002445  0.004773  0.007922 



In [55]:
def plot_svi_coefficients(svi_sample, title, xlim  = .05, ylim = .05):

    site_p_svi = svi_sample['p'].flatten().tolist()
    site_t_svi = svi_sample['t'].flatten().tolist()
  
    source_svi = ['svi'] * len(site_p_svi)

    data = {
        'p': site_p_svi ,
        't': site_t_svi ,
        'inference_method': source_svi 
    }

    df = pd.DataFrame(data)

    fig = px.scatter(df, x='p', y='t', 
                    title= title,
                    template='plotly_dark')

    fig.update_xaxes(range=[-xlim, xlim])
    fig.update_yaxes(range=[-ylim, ylim])

    fig.update_layout(autosize=False,
            width=600,
            height=600
    )

    fig.update_traces(marker=dict(size=3.5, opacity=0.4))

    fig.show()

plot_svi_coefficients(locust_sigmavar, 'Locust(posterior samples)')

In [18]:
#communication needs debugging if we want to apply it to locust data

info_time_decay=10,
info_spatial_decay=0.15

com = ft.generate_communicates(sim, info_time_decay, info_spatial_decay)
       
sim.communicatesDF = com["communicatesDF"]

TypeError: can only concatenate tuple (not "int") to tuple